In [1]:
!pip install dash


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 18.0 MB/s eta 0:00:00


In [4]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import yfinance as yf
import pandas as pd

# Define asset categories based on your grouping
hardware_tickers = ['AAPL', 'NVDA', 'INTC']
software_tickers = ['MSFT', 'ORCL', 'ADBE']
digital_media_tickers = ['GOOGL', 'META', 'NFLX']
crypto_tickers = ['BTC-USD', 'DOGE-USD', 'ETH-USD', 'SOL-USD', 'ADA-USD']

tickers = hardware_tickers + software_tickers + digital_media_tickers + crypto_tickers

# Download data from yfinance
data = yf.download(tickers, period="5d", interval="1d")
data = data.ffill().bfill()

# Calculate percent change relative to the opening price of the previous day
pct_change_open_close = (data['Adj Close'].iloc[-1] - data['Open'].iloc[-1]) / data['Open'].iloc[-1] * 100
adj_close = data['Adj Close'].iloc[-1]

# Create a DataFrame
df = pd.DataFrame({
    'Ticker': adj_close.index,
    'Price': adj_close.values,
    'Percent Change': pct_change_open_close.values
})

# Calculate market cap and add it to the DataFrame
df['Market Cap'] = df['Ticker'].apply(lambda x: yf.Ticker(x).info.get('marketCap', 0))

# Add sector information for each asset
df['Sector'] = df['Ticker'].apply(lambda x: 'Hardware' if x in hardware_tickers else
                                  'Software' if x in software_tickers else
                                  'Digital Media' if x in digital_media_tickers else
                                  'Cryptocurrency')

# Initialize the Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='treemap'),
    html.Div(id='stock-details', style={
        'backgroundColor': '#2d2d2d',
        'color': 'white',
        'padding': '20px',
        'borderRadius': '10px',
        'boxShadow': '2px 2px 10px rgba(0,0,0,0.3)',
        'marginTop': '20px',
        'fontFamily': 'Arial, sans-serif',
        'display': 'flex',
        'flexWrap': 'wrap',
        'justifyContent': 'space-around'
    })
])

# Define the callback to update the treemap
@app.callback(
    [Output('treemap', 'figure'),
     Output('stock-details', 'children')],
    [Input('treemap', 'clickData')]
)
def update_output(clickData):
    # Create treemap using market cap as the size and display percent change in the center of each block
    fig = px.treemap(df, path=['Sector', 'Ticker'], values='Market Cap',
                     hover_data={'Price': ':.2f', 'Percent Change': ':.2f%', 'Market Cap': ':.2f'},
                     color='Percent Change',
                     color_continuous_scale='RdYlGn',
                     title="Market Heatmap with Percent Change in Center")

    # Add percent change as text in the center of each block
    fig.update_traces(texttemplate='%{label}<br>%{customdata[1]:.2f}%', textposition='middle center')

    # Remove parent and id from the hover tooltip
    fig.update_traces(hovertemplate='<b>%{label}</b><br>Market Cap: %{value}<br>Price: %{customdata[0]:.2f}<br>Percent Change: %{customdata[1]:.2f}%')

    # Display stock details on click
    if clickData:
        ticker = clickData['points'][0]['label']
        stock_info = yf.Ticker(ticker).info
        details = [
            html.Div(f"{key.replace('_', ' ').title()}: {value}", style={'margin': '10px', 'minWidth': '150px', 'textAlign': 'center', 'border': '1px solid #aaa', 'padding': '10px', 'borderRadius': '5px', 'boxShadow': '1px 1px 5px rgba(0,0,0,0.2)'})
            for key, value in stock_info.items() if key in ['marketCap', 'trailingPE', 'forwardPE', 'trailingEps', 'dividendYield', 'beta', 'fiftyTwoWeekHigh', 'fiftyTwoWeekLow', 'volume']
        ]
        return fig, details

    return fig, "Click on an element to see details."

# Run the server
if __name__ == '__main__':
    app.run_server(debug=True)


[*********************100%***********************]  14 of 14 completed


<IPython.core.display.Javascript object>